<a href="https://colab.research.google.com/github/JaeyeongLee1/DSAIL_summer_internship/blob/main/PMF_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [213]:
import numpy as np
import pandas as pd
import math
from google.colab import drive

drive.mount('/content/drive')

ratings = pd.read_csv('/content/drive/MyDrive/랩인턴/ratings.csv') #movielens data 사용 https://www.kaggle.com/sengzhaotoo/movielens-small?select=README.txt
movies = pd.read_csv('/content/drive/MyDrive/랩인턴/movies.csv')
del ratings['timestamp']
del movies['genres']
df = pd.merge(ratings,movies)

#N = len(set(df['userId']))
N=50 #batch size
#M = len(set(df['movieId']))
M=50 #batch size
D = 5

users = list(set(df['userId']))[:N]
movies = list(set(df['movieId']))[:M]

U = np.random.rand(N,D)
V = np.random.rand(M,D)
R = np.zeros((N,M))
for i in range(N):
  for j in range(M):
    s1=df[df['userId']==users[i]]
    s=s1[s1['movieId']==movies[j]]
    x = s.rating.values
    if len(x)==0:
      R[i][j]=None
    else:
      R[i][j]=x[0]

R=(R-0.5)/4.5
def g(x):
  return 1/(1+math.exp(-x))
def g_prime(x):
  return math.exp(-x)/((1+math.exp(-x))**2)

def RMSE(U,V):
  RMSE=0
  for i in range(N):
    for j in range(M):
      if not np.isnan(R[i][j]):
        RMSE += (R[i][j]-g(U[i].dot(V[j].T)))**2
  return RMSE

def E(U,V):
  E=0
  for i in range(N):
    E += 0.5*lambda_u*U[i].dot(U[i].T)
  for j in range(M):
    E += 0.5*lambda_v*V[j].dot(V[j].T)
  for i in range(N):
    for j in range(M):
      if not np.isnan(R[i][j]):
        E += 0.5*(R[i][j]-g(U[i].dot(V[j].T)))**2
  return E

def gradE(U,V):
  gradE_U,gradE_V = np.zeros((N,D)),np.zeros((M,D))
  for i in range(N):
    for j in range(M):
      if not np.isnan(R[i][j]):
        gradE_U[i] += (R[i][j]-g(U[i].dot(V[j].T)))*g_prime(U[i].dot(V[j].T))*(-V[j])
    gradE_U[i] += lambda_u*U[i]
  for j in range(M):
    for i in range(N):
      if not np.isnan(R[i][j]):
        gradE_V[j] += (R[i][j]-g(U[i].dot(V[j].T)))*g_prime(U[i].dot(V[j].T))*(-U[i])
    gradE_V[j] += lambda_v*V[j]
  gradE = (gradE_U,gradE_V)
  return gradE



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [214]:
lambda_u = 0.01
lambda_v = 0.001
learning_rate=0.05
epoch=1000
print(RMSE(U,V))
for i in range(epoch):
  U=U-learning_rate*gradE(U,V)[0]
  V=V-learning_rate*gradE(U,V)[1]
print(RMSE(U,V))

11.32952209455415
0.46906236526915746
